In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
import numpy as np
from tensorflow.python.summary.summary_iterator import summary_iterator
import tensorflow as tf
from glob import glob
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import Counter
import datetime
import math


In [2]:
results_path = "./models/Reinforce_One_Dataset/"
datasets = list(map(os.path.basename, glob(r"OneDatasetLearning/Classification/*")))

In [3]:
all_features = []
all_sizes = []

for curr_d in datasets:
    s, f = pd.read_csv(f'OneDatasetLearning/Classification/{curr_d}/{curr_d}.csv').shape
    all_sizes.append(s)
    all_features.append(f)


In [4]:
dataset_to_feature_size = dict([(x, all_features[idx]) for idx, x in enumerate(datasets)])
dataset_to_size = dict([(x, all_sizes[idx]) for idx, x in enumerate(datasets)])

In [5]:
def remove_min_max(l):
    l.remove(min(l))
    l.remove(max(l))

# NEON


In [6]:
all_logs_folders = glob("runs2/*")

In [7]:
all_runs_lambdas = {
#     'NEON 4': lambda l: list(filter(lambda x: 'with_loop' in x and
#                                                'Random_Actions' not in x and
#                                                'single_test' not in x and
#                                                'warmup' not in x, l)),
    'NEON 4': lambda l: list(filter(lambda x: 'with_loop' in x and
                                         'Random_Actions' not in x and
                                         'single_test' not in x and
                                         'warmup' in x, l)),
#     'NEON 1': lambda l: list(filter(lambda x: 'with_loop' not in x and 
#                                      'Random_Actions' not in x and 
#                                      'LAP' not in x and
#                                      'pruning' not in x and
#                                      'Combined' not in x and
#                                      'ADMM' not in x and
#                                      'AMC' not in x and
#                                      'warmup' not in x, l)),    
    'NEON 1': lambda l: list(filter(lambda x: 'with_loop' not in x and 
                                         'Random_Actions' not in x and 
                                         'LAP' not in x and
                                         'pruning' not in x and
                                         'Combined' not in x and
                                         'ADMM' not in x and
                                         'AMC' not in x and
                                         'warmup' in x, l))
}

In [8]:
titles = list(map(lambda x: f'{x} ({dataset_to_size[x]})', datasets))

In [9]:
def get_delta_by_log(log_paths):
# log_paths = glob(os.path.join(a[0], "*"))
    min_of_all_logs = datetime.datetime.max
    max_of_all_logs = datetime.datetime.min
    max_log = None

    for log_path in log_paths:    
        curr_log = list(summary_iterator(log_path))
        if len(curr_log) == 1:continue
        last_log = datetime.datetime.fromtimestamp(curr_log[-1].wall_time)
        first_log = datetime.datetime.fromtimestamp(curr_log[0].wall_time)

        if min_of_all_logs > first_log:
            min_of_all_logs = first_log

        if max_of_all_logs < last_log:
            max_of_all_logs = last_log
            max_log = curr_log[-1]


    delta_last_to_first = (max_of_all_logs - min_of_all_logs).total_seconds() / 60 / 60
    average_evaluation_time = (delta_last_to_first * 60) / max_log.step
    return delta_last_to_first, average_evaluation_time

In [10]:
def add_value_to_correct_lst(small, medium, large, value, ds):
    if dataset_to_size[ds] <= 5000:
        small.append(value)
    elif dataset_to_size[ds] <= 30000:
        medium.append(value)
    else:
        large.append(value)

In [11]:
all_data = []

In [12]:
all_hyper_params_possibilities = ['0', '1', '5', '50']

for name, filter_func in all_runs_lambdas.items():
    small_training = []
    medium_training = []
    large_training = []
    
    small_evaluation = []
    medium_evaluation = []
    large_evaluation = []    
    relevant_logs = filter_func(all_logs_folders)

    for idx, curr_acc_hp in enumerate(all_hyper_params_possibilities):        
        curr_group = list(filter(lambda x: re.search(r'acc_reduction_(\d{1,2})', x).group(1) == curr_acc_hp, 
                                 relevant_logs))        
        for curr_ds in datasets:
            curr_log_folder = list(filter(lambda x: curr_ds in x, curr_group))[0]
            log_path = glob(os.path.join(curr_log_folder, "*"))
            curr_delta = get_delta_by_log(log_path)
            add_value_to_correct_lst(small_training, medium_training, large_training, curr_delta[0], curr_ds)            
            add_value_to_correct_lst(small_evaluation, medium_evaluation, large_evaluation, curr_delta[1], curr_ds)
        

    for _ in range(4):
        remove_min_max(small_training)
        remove_min_max(medium_training)
        remove_min_max(large_training)
        remove_min_max(small_evaluation)
        remove_min_max(medium_evaluation)
        remove_min_max(large_evaluation)

    all_data.append([f'{name}', np.average(small_training), np.average(small_evaluation),
                    np.average(medium_training), np.average(medium_evaluation),
                    np.average(large_training), np.average(large_evaluation)])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


# ADMM

In [13]:
time_folder = r"./times"
all_times = glob(os.path.join(time_folder, "*"))

times_path = list(filter(lambda x: "ADMM" in x, all_times))[0]
time_df = pd.read_csv(times_path)

time_df['ds_size'] = time_df.Dataset.apply(dataset_to_size.get)
time_df['time_in_hours'] = time_df.time / 60 / 60

small_times = time_df[time_df.ds_size <= 5000]
medium_times = time_df[(time_df.ds_size > 5000) & (time_df.ds_size <= 30000)]
large_times = time_df[time_df.ds_size > 30000]

all_data.append([f'ADMM', "", small_times.time_in_hours.mean(),
                "", medium_times.time_in_hours.mean(),
                "", large_times.time_in_hours.mean()])

# LAP 1

In [14]:
time_folder = r"./times"
all_times = glob(os.path.join(time_folder, "*"))

times_path = list(filter(lambda x: "LAP2_1" in x, all_times))[0]
time_df = pd.read_csv(times_path)

time_df['ds_size'] = time_df.Dataset.apply(dataset_to_size.get)
time_df['time_in_hours'] = time_df.time / 60 / 60

small_times = time_df[time_df.ds_size <= 5000]
medium_times = time_df[(time_df.ds_size > 5000) & (time_df.ds_size <= 30000)]
large_times = time_df[time_df.ds_size > 30000]

all_data.append([f'LAP 1', "", small_times.time_in_hours.mean(),
                "", medium_times.time_in_hours.mean(),
                "", large_times.time_in_hours.mean()])

# LAP 4

In [15]:
time_folder = r"./times"
all_times = glob(os.path.join(time_folder, "*"))

times_path = list(filter(lambda x: "LAP2_4" in x, all_times))[0]
time_df = pd.read_csv(times_path)

time_df['ds_size'] = time_df.Dataset.apply(dataset_to_size.get)
time_df['time_in_hours'] = time_df.time / 60 / 60

small_times = time_df[time_df.ds_size <= 5000]
medium_times = time_df[(time_df.ds_size > 5000) & (time_df.ds_size <= 30000)]
large_times = time_df[time_df.ds_size > 30000]

all_data.append([f'LAP 4', "", small_times.time_in_hours.mean(),
                "", medium_times.time_in_hours.mean(),
                "", large_times.time_in_hours.mean()])

# LAP 10

In [16]:
time_folder = r"./times"
all_times = glob(os.path.join(time_folder, "*"))

times_path = list(filter(lambda x: "LAP2_10" in x, all_times))[0]
time_df = pd.read_csv(times_path)

time_df['ds_size'] = time_df.Dataset.apply(dataset_to_size.get)
time_df['time_in_hours'] = time_df.time / 60 / 60

small_times = time_df[time_df.ds_size <= 5000]
medium_times = time_df[(time_df.ds_size > 5000) & (time_df.ds_size <= 30000)]
large_times = time_df[time_df.ds_size > 30000]

all_data.append([f'LAP 10', "", small_times.time_in_hours.mean(),
                "", medium_times.time_in_hours.mean(),
                "", large_times.time_in_hours.mean()])

# pruning

In [17]:
time_folder = r"./times"
all_times = glob(os.path.join(time_folder, "*"))

times_path = list(filter(lambda x: "pruning" in x, all_times))[0]
time_df = pd.read_csv(times_path)

time_df['ds_size'] = time_df.Dataset.apply(dataset_to_size.get)
time_df['time_in_hours'] = time_df.time / 60 / 60

small_times = time_df[time_df.ds_size <= 5000]
medium_times = time_df[(time_df.ds_size > 5000) & (time_df.ds_size <= 30000)]
large_times = time_df[time_df.ds_size > 30000]

all_data.append([f'Pruning', "", small_times.time_in_hours.mean(),
                "", medium_times.time_in_hours.mean(),
                "", large_times.time_in_hours.mean()])

In [18]:
pd.DataFrame(all_data)

,0,1,2,3,4,5,6
0,NEON 4,3.39711,0.651204,25.3498,4.859380,57.9172,13.324788
1,NEON 1,0.556331,0.106645,4.17152,0.799652,11.9874,2.297901
2,ADMM,,0.011637,,0.111004,,0.264999
3,LAP 1,,0.085758,,0.085931,,0.087914
4,LAP 4,,0.085529,,0.087503,,0.090471
5,LAP 10,,0.085087,,0.087004,,0.088746
6,Pruning,,0.030185,,0.360345,,0.793287


# AMC 10 iterations

In [19]:
time_folder = r"./times"
all_times = glob(os.path.join(time_folder, "*"))

times_path = list(filter(lambda x: "AMC" in x, all_times))

In [20]:
all_path_datasets = list(map(lambda x: re.search("10iters_(.*).csv" ,x).group(1), times_path))

In [21]:
dataset_column = np.array(list(zip(all_path_datasets, all_path_datasets))).flatten()

In [22]:
all_amc_data = pd.concat(list(map(pd.read_csv,times_path)))
all_amc_data['ds'] = dataset_column
all_amc_data['ds_size'] = all_amc_data.ds.apply(dataset_to_size.get)

In [23]:
training_amc_data = all_amc_data[all_amc_data.Dataset == 'train']
eval_amc_data = all_amc_data[all_amc_data.Dataset == 'eval']

In [24]:
training_amc_data['time_in_minutes'] = training_amc_data.time / 60 / 60
eval_amc_data['time_in_hours'] = eval_amc_data.time / 60

<ipython-input-24-ddd4a1b021e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_amc_data['time_in_minutes'] = training_amc_data.time / 60 / 60
<ipython-input-24-ddd4a1b021e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_amc_data['time_in_hours'] = eval_amc_data.time / 60


In [25]:
training_small_times = training_amc_data[training_amc_data.ds_size <= 5000]
training_medium_times = training_amc_data[(training_amc_data.ds_size > 5000) & (training_amc_data.ds_size <= 30000)]
training_large_times = training_amc_data[training_amc_data.ds_size > 30000]

eval_small_times = eval_amc_data[eval_amc_data.ds_size <= 5000]
eval_medium_times = eval_amc_data[(eval_amc_data.ds_size > 5000) & (eval_amc_data.ds_size <= 30000)]
eval_large_times = eval_amc_data[eval_amc_data.ds_size > 30000]

In [26]:
all_data.append([f'AMC 10 iters', training_small_times.time_in_minutes.mean(), eval_small_times.time_in_hours.mean(),
                training_medium_times.time_in_minutes.mean(), eval_medium_times.time_in_hours.mean(),
                training_large_times.time_in_minutes.mean(), eval_large_times.time_in_hours.mean()])

In [27]:
pd.DataFrame(all_data)

,0,1,2,3,4,5,6
0,NEON 4,3.39711,0.651204,25.3498,4.859380,57.9172,13.324788
1,NEON 1,0.556331,0.106645,4.17152,0.799652,11.9874,2.297901
2,ADMM,,0.011637,,0.111004,,0.264999
3,LAP 1,,0.085758,,0.085931,,0.087914
4,LAP 4,,0.085529,,0.087503,,0.090471
5,LAP 10,,0.085087,,0.087004,,0.088746
6,Pruning,,0.030185,,0.360345,,0.793287
7,AMC 10 iters,0.239506,0.026960,0.800172,0.241931,1.20739,0.755098


In [28]:
pd.DataFrame(all_data).to_csv("times comparison.csv")